In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd

In [5]:
#df = pd.read_csv('drive/MyDrive/uh/out.digg-friends', sep = ' |\t', skiprows=[0], names = ['from', 'to', 'weight', 'timestamp'])

Считываем данные в таблицу, колонками которой являются вершина 1, вершина 2, вес, время появления ребра между вершинами


In [6]:
!ls 'drive/MyDrive/uh'

dnc-corecipient.cvc		out.radoslaw_email_email
email-Eu-core-temporal.txt.cvc	out.soc-sign-bitcoinalpha
opsahl-ucsocial.cvc		out.soc-sign-bitcoinotc
out.digg-friends		out.sx-askubuntu
out.dnc-corecipient		out.sx-mathoverflow
out.email-Eu-core-temporal.txt	out.sx-superuser
out.munmun_digg_reply		radoslaw_email_email.cvc
out.opsahl-ucsocial		soc-sign-bitcoinotc.cvc
out.prosper-loans


In [7]:
list_of_name = ["opsahl-ucsocial", "radoslaw_email_email","soc-sign-bitcoinotc","dnc-corecipient", 'out.email-Eu-core-temporal.txt']

In [8]:
#list_of_name = ['out.digg-friends','out.prosper-loans','out.sx-askubuntu','out.dnc-corecipient','out.soc-sign-bitcoinalpha','out.sx-mathoverflow','out.munmun_digg_reply','out.soc-sign-bitcoinotc','out.sx-superuser']

Датасеты разного типа считаем по-разному


In [9]:
df1 = pd.read_csv("drive/MyDrive/uh/out.opsahl-ucsocial", sep = ' |\t', skiprows=2, names = ['v1', 'v2', 'weight', 'timestamp'])
df2 = pd.read_csv("drive/MyDrive/uh/out.radoslaw_email_email", sep = '  | ', skiprows=2, names = ['v1', 'v2','weight', 'timestamp'])
df3 = pd.read_csv("drive/MyDrive/uh/out.soc-sign-bitcoinotc", sep = ' |\t', skiprows=1, names = ['v1', 'v2', 'weight', 'timestamp'])
df4 = pd.read_csv("drive/MyDrive/uh/out.dnc-corecipient", sep = ' |\t', skiprows=1, names = ['v1', 'v2', 'timestamp'])
df5 = pd.read_csv('drive/MyDrive/uh/out.email-Eu-core-temporal.txt', sep = ' |\t',  names = ['v1', 'v2', 'timestamp'])

<ipython-input-9-408a258bb22c>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df1 = pd.read_csv("drive/MyDrive/uh/out.opsahl-ucsocial", sep = ' |\t', skiprows=2, names = ['v1', 'v2', 'weight', 'timestamp'])
<ipython-input-9-408a258bb22c>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df2 = pd.read_csv("drive/MyDrive/uh/out.radoslaw_email_email", sep = '  | ', skiprows=2, names = ['v1', 'v2','weight', 'timestamp'])
<ipython-input-9-408a258bb22c>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' 

In [10]:
# for i in range(len(list_of_name)):
#     filename = 'drive/MyDrive/uh/' + 'out.' + list_of_name[i]
#     print(filename)
#     locals()[f'df{i+1}'] = pd.read_csv(filename, sep = ' |\t', skiprows=[0], names = ['v1', 'v2', 'weight', 'timestamp'])

Выведем первые пять элементов одной из таблиц, чтобы убедиться что всё считалось верно


In [11]:
df1.head(5)

,v1,v2,weight,timestamp
0,1,2,1,1082008561
1,3,4,1,1082123439
2,5,2,1,1082381991
3,6,7,1,1082407219
4,8,7,1,1082407356


In [12]:
#df.head(10)

**1. Считаем число вершин, число рёбер, плотность (отношение числа рёбер к максимально
возможному числу рёбер), число компонент слабой связности, долю вершин в
максимальной по мощности компоненте слабой связности.**

In [13]:
def task_1(df):
    vertexes = (set(df.v2.unique()) | set(df.v1.unique()))
    number_of_vertexes = len(vertexes)
    number_of_edges_directed =df.shape[0] 
    edges = sorted(set(zip((df[['v1','v2']]).min(axis = 1), (df[['v1','v2']]).max(axis = 1))))
    number_of_edges_undirected = len(edges)
    density = len(edges) / ((number_of_vertexes*(number_of_vertexes - 1))/2)
    print("Число вершин: ", number_of_vertexes)
    print("Число направленных рёбер: ", number_of_edges_directed)
    print("Число ненаправленных рёбер: ", number_of_edges_undirected)
    print("Плотность: ", density)
    return vertexes, number_of_vertexes, number_of_edges_directed, edges, number_of_edges_undirected, density

In [18]:
from collections import defaultdict
from collections import deque

In [19]:
def dfs_and_gcc(edges, neighbours, vertexes, max_component=False):
#на вход подается неориентированный список смежности
    access_vertexes = deque()
    visited = set()
    connected_components = defaultdict(set)
    connected_components_number = 0
    for vertex in vertexes:
        if vertex not in visited:
            connected_components_number += 1
            access_vertexes.append(vertex)
            while access_vertexes:
                u = access_vertexes.pop()
                for v in neighbours[u]:
                    if v not in visited:
                        access_vertexes.append(v)
                        visited.add(v)
                connected_components[connected_components_number].add(u)
                visited.add(u)
    print("Число компонент слабой связности: ", connected_components_number)
    if max_component:
        ind_max_component = max(connected_components, key=lambda num: len(connected_components[num]))
        return connected_components_number, ind_max_component, connected_components
    return connected_components_number, connected_components

In [20]:
i = 0
for df in [df1,df2,df3,df4,df5]:
  print("Название датасета: ", list_of_name[i])
  i +=1
  vertexes, number_of_vertexes, number_of_edges_directed, edges, number_of_edges_undirected, density = task_1(df)
  neighbours = defaultdict(set)
  for edge in edges:
    if edge[0] != edge[1]:
      neighbours[edge[0]].add(edge[1])
      neighbours[edge[1]].add(edge[0])
  connected_components_number, ind_max_component, connected_components = dfs_and_gcc(edges, neighbours, vertexes, True)
  print("Доля вершин в максимальной компоненте: ", len(connected_components[ind_max_component])/ number_of_vertexes)
  print()

Название датасета:  opsahl-ucsocial
Число вершин:  1899
Число направленных рёбер:  59835
Число ненаправленных рёбер:  13838
Плотность:  0.007678601848568738
Число компонент слабой связности:  4
Доля вершин в максимальной компоненте:  0.9968404423380727

Название датасета:  radoslaw_email_email
Число вершин:  167
Число направленных рёбер:  82927
Число ненаправленных рёбер:  3251
Плотность:  0.23454296226823462
Число компонент слабой связности:  1
Доля вершин в максимальной компоненте:  1.0

Название датасета:  soc-sign-bitcoinotc
Число вершин:  5881
Число направленных рёбер:  35592
Число ненаправленных рёбер:  21492
Плотность:  0.001243020588612932
Число компонент слабой связности:  4
Доля вершин в максимальной компоненте:  0.9989797653460296

Название датасета:  dnc-corecipient
Число вершин:  906
Число направленных рёбер:  12085
Число ненаправленных рёбер:  10429
Плотность:  0.025438756967058163
Число компонент слабой связности:  25
Доля вершин в максимальной компоненте:  0.93708609271